# Scipp neutron data reduction demo

## Introduction

In this notebook we use the same datafiles as in `MantidDataTutorial.ipynb`. But the data reduction is done using Scipp. Its online documentation is available at [scipp.github.io](https://scipp.github.io/).   
The datafiles can be downloaded from [dropbox](https://www.dropbox.com/scl/fo/kduv87wx4j2cc71u9lu97/AKy4Yn7Q_EAvXyd1jV0jd1w?rlkey=jcy7759n02vqc8ikwyglr5k30&st=rvf18jlv&dl=0).

Some basic familiarity with python is assumed. See this [link](https://ess-dmsc-dram.github.io/dmsc-school/1-python/python_basics.html) for a short introduction if needed.  

In a Jupyter notebook, to execute the content of one of the cells, just select it and hit `shift-enter` or click on the `play` icon in the toolbar! More information can be found at this [link](https://jupyterlab.readthedocs.io/en/stable/user/interface.html).

## Import Scipp and some other tools

As with most python scripts, our script will begin with importing the necessary modules.  

Just select the cell below and hit `Shift-Enter` to run it.  

If you see any errors at this stage, check your conda environment is activated.

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn
import plopp as pp
import scippnexus as snx
import matplotlib.pyplot as plt

# mute mantid log display
from mantid.kernel import ConfigService
ConfigService.setLogLevel(1)

%matplotlib widget

## Path to the data

In [ ]:
dataPath = "/Users/66j/Downloads"
dataFile = "NIST_Silicon"
dataSet = f"{dataPath}/{dataFile}.nxs"

## Load the data 

Loading Nexus files requires [Mantid](https://www.mantidproject.org).
See, e.g., [Installation](https://scipp.github.io/getting-started/installation.html) on how to install scipp and Mantid with `conda`.

In [ ]:
silicon = scn.load_with_mantid(dataSet, 
                               load_pulse_times=False)

## Inspect the data



In [ ]:
silicon

It is a `DataGroup` containing metadata and detector data (neutron).  

Extract the actual events:

In [ ]:
si_da = silicon['data']
si_da

In [ ]:
sc.show(si_da)

The underlying events can be inspected by using: 

In [ ]:
si_da.bins.constituents['data']

## Instrument view

In [ ]:
pp.scatter3d(si_da.hist(), 
             pos='position', 
             pixel_size=0.01, 
             cbar=True)

## Convert units

For more information on this topic, please refer to [Scipp documentation](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)

In [ ]:
dspacing_graph = {
    **scn.conversion.graph.beamline.beamline(scatter=True),
    **scn.conversion.graph.tof.elastic_dspacing('tof'),
}
sc.show_graph(dspacing_graph)

In [ ]:
si_dsp = si_da.transform_coords('dspacing', 
                                graph=dspacing_graph)
si_dsp

## Plot selected spectra

In [ ]:
sp2plot1 = 0
sp2plot2 = 18431

assert sp2plot1 in range(si_dsp.sizes['spectrum']), f"{sp2plot1} out of range"
assert sp2plot2 in range(si_dsp.sizes['spectrum']), f"{sp2plot2} out of range"

pp.plot({f'Sp{sp2plot1}': si_dsp['spectrum', sp2plot1].hist(dspacing=500),
         f'Sp{sp2plot2}': si_dsp['spectrum', sp2plot2].hist(dspacing=500)}, 
        errorbars=False, grid=True)

## Diffraction Focussing
### Create groups

Contrary to `Mantid`, `Scipp` does not rely on Instrument Definition Files to load the data. The geometry of the instrument is available via $L_1$, $L_2$, source and sample positions and the detectors' positions.

The grouping scheme has to be defined manually. 

We are going to create 6 pixel groups according to the vertical columns of modules in each detector. 

In [ ]:
panel_size = 32**2
si_dsp.coords['columns'] = sc.array(dims=['spectrum'], 
                                   values=[int(i//(3*panel_size) + 1) if i<3*3*panel_size else int(9 - i//(3*panel_size)) for i in range(si_dsp.sizes['spectrum'])],
                                   unit=None)
si_dsp

Using the instrument view, we can quickly check that the selection 
by spectra corresponds to the correct column:

In [ ]:
pp.scatter3d({'Column1': si_dsp['spectrum', :3*panel_size].hist(),
              'Column3': si_dsp['spectrum', 6*panel_size: 9*panel_size].hist()},
             pos='position', 
             pixel_size=0.01,
             title='Selection of columns of detectors',
             cbar=True)

Below we check how the groups are distributed among the `spectra`:

In [ ]:
si_dsp.coords['columns'].plot(
    grid=True,
    title='Check distribution of groups',
    linestyle='solid',
    marker=''
)

We check that the spectra are equally distributed among the groups:

In [ ]:
len(set([(si_dsp.coords['columns'] == sc.scalar(i, unit=None)).sum().value
 for i in range(1, 7)])) == 1

We check the evolution of $2\theta$ in comparison with the columns:

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)

p1 = pp.plot({'two theta': sc.to_unit(si_dsp.coords['two_theta'], unit='deg')}, 
             grid=True, 
             linestyle='', 
             marker='.', 
             ax=ax[0])

p2 = pp.plot({'column': si_dsp.coords['columns']}, 
             grid=True, 
             linestyle='solid', 
             marker='',
             ax=ax[1])

The next step is to combine these groups:

In [ ]:
si_column = si_dsp.group('columns')
si_column

We now have a new DataArray `si_column` with 6 groups in the `columns` coordinate.   
Now we plot all 6 together: 

In [ ]:
pp.plot({f'Column {i}': si_column['columns', sc.scalar(i, unit=None)].hist(dspacing=500)['dspacing', 2:] for i in range(1, 7)},
        grid=True, errorbars=False)

In [ ]:
dspacing_bins = sc.geomspace(dim='dspacing', 
                              unit='angstrom', 
                              start=0.3,
                              stop=5.26,
                              num=3582) # number from implementation in Mantid

si_column_bin = si_column.bin(dspacing=dspacing_bins)
si_column_bin

In [ ]:
dg_to_plot = sc.DataGroup()
for i in range(1, 7):
    dg_to_plot[f'Column {i}'] = si_column_bin['columns', sc.scalar(i, unit=None)].hist() 
pp.plot(dg_to_plot, grid=True, errorbars=False)

The event data are better sampled.  
These plots already exhibit some of the properties of TOF detectors. 
We can see that: 

1. The maximum d-spacing increases towards low angle
2. The minimum d-spacing decreases towards high angle
3. The diffraction resolution (Bragg peak width) increases towards higher values.

But if we zoom in to, say the two peaks at 3.1 Å, we will see that they don't quite line up. Since these peaks correspond to the same Bragg reflection, they should have the same d-spacing. This is due to errors in the default mathematical description of the instrument used when calculating the conversion from TOF to d-spacing. To correct for this, we must apply a calibration. This calibration was already measured, and the calibration values (the "Diffractometer Constants") are stored in the provided file `diffract_consts.h5`. 

In [ ]:
dsp_min = 3 
dsp_max = 3.5

pp.plot(dg_to_plot['dspacing', dsp_min*sc.Unit('angstrom'):dsp_max*sc.Unit('angstrom')],
        title=f'Zoom in dspacing [{dsp_min}, {dsp_max}] Angstrom',
        grid=True, errorbars=False)

Below we start from the original DataArray (in `ToF`). 
We apply the calibration and convert to d-spacing again:

## Convert to d-spacing with calibration
### Define graph

In [ ]:
si_da

In [ ]:
calib = sc.DataGroup()



with snx.File(f"{dataPath}/diffract_consts.h5") as f: 
    calib = f['calibration'][()]

calib

In [ ]:
def dspacing_with_calibration(tof, difc):
    difc = sc.reciprocal(difc)
    return difc * tof

calibration_graph = {"dspacing": dspacing_with_calibration}
sc.show_graph(calibration_graph)

### Convert units

In [ ]:
si_da.coords['difc'] = sc.array(dims=['spectrum'], 
                                values=calib['difc'].values, 
                                unit=sc.Unit('us/Angstrom'))

si_dsp_calib = si_da.transform_coords("dspacing", 
                                      graph=calibration_graph)
si_dsp_calib

## Group

In [ ]:
si_dsp_calib.coords['columns'] = si_dsp.coords['columns']
si_dsp_calib_gp = si_dsp_calib.group('columns')
si_dsp_calib_gp

In [ ]:
pp.plot({f'Column {i}': si_dsp_calib_gp['columns', sc.scalar(i, unit=None)].hist(dspacing=400) for i in range(1, 7)},
        grid=True, errorbars=False)

## Bin

In [ ]:
dspacing_bins1 = sc.geomspace(dim='dspacing', 
                              unit='angstrom', 
                              start=0.35,
                              stop=4.7,
                              num=3582) # number from implementation in Mantid

si_dsp_calib_gp_bin = si_dsp_calib_gp.bin(dspacing=dspacing_bins1)
si_dsp_calib_gp_bin

If now we zoom in to the two peaks at 3.1 Å, we will see that they line up:

In [ ]:
dsp_min = 3 
dsp_max = 3.5

pp.plot({f'Column {i}': si_dsp_calib_gp_bin['columns', sc.scalar(i, unit=None)].hist()['dspacing', dsp_min*sc.Unit('angstrom'):dsp_max*sc.Unit('angstrom')] 
         for i in range(1, 7)},
        title=f'Zoom in dspacing [{dsp_min}, {dsp_max}] Angstrom',
        grid=True, errorbars=False)

## Removing events

Before removing the events, we rebin the data to a grid which will also be used for the Vanadium data:

In [ ]:
dspacing_bins2 = sc.geomspace(dim='dspacing', 
                              unit='angstrom', 
                              start=0.3,
                              stop=5.26,
                              num=3582) # number from implementation in Mantid

si_dsp_calib_gp_hist = si_dsp_calib_gp_bin.hist(dspacing=dspacing_bins2)

## Normalising the data
### Loading and reducing Vanadium and Empty data

In [ ]:
empty = scn.load_with_mantid(f"{dataPath}/empty.nxs")['data']
van = scn.load_with_mantid(f"{dataPath}/vanadium-niobium.nxs")['data']

# convert to d-spacing with calibration
empty.coords['difc'] = sc.array(dims=['spectrum'], 
                                values=calib['difc'].values, 
                                unit=sc.Unit('us/Angstrom'))
van.coords['difc'] = sc.array(dims=['spectrum'], 
                                values=calib['difc'].values, 
                                unit=sc.Unit('us/Angstrom'))

empty_dsp = empty.transform_coords("dspacing",
                                   graph=calibration_graph)

van_dsp = van.transform_coords("dspacing",
                               graph=calibration_graph)

# create groups
assert empty_dsp.sizes['spectrum'] == si_dsp.sizes['spectrum']
empty_dsp.coords['columns'] = si_dsp.coords['columns']

assert van_dsp.sizes['spectrum'] == si_dsp.sizes['spectrum']
van_dsp.coords['columns'] = si_dsp.coords['columns']

In [ ]:
# group
empty_gp = empty_dsp.group('columns')

van_gp = van_dsp.group('columns')

In [ ]:
# rebin
empty_bin = empty_gp.bin(dspacing=dspacing_bins2)

van_bin = van_gp.bin(dspacing=dspacing_bins2)

In [ ]:
pp.plot({f'Column {i}': empty_bin['columns', sc.scalar(i, unit=None)].hist() for i in range(1, 7)}, 
        grid=True,  title='Empty', errorbars=False)

In [ ]:
pp.plot({f'Column {i}': van_bin['columns', sc.scalar(i, unit=None)].hist() for i in range(1, 7)}, 
        grid=True,  title='Vanadium', errorbars=False)

The normalisation correction is captured in the vanadium-niobium scattering so we want to remove any other "background" contribution to that measurement. This is why we collected the "empty" instrument dataset, which should be identical to the vanadium-niobium measurement (e.g., same detector positions and same front end settings) only without the V-Nb. The vanadium-niobium signal we need is obtained by subtracting this empty:

## Subtract empty and convert to histogram

In [ ]:
norm_column = sc.hist(van_bin.bins.concatenate(-empty_bin))
norm_column

In [ ]:
pp.plot({f'Column {i}': norm_column['columns', sc.scalar(i, unit=None)] for i in range(1, 7)}, 
        grid=True,  title='Vanadium-Empty', errorbars=False)

The next processing step is to smooth the data. 

The normalisation correction is slowly varying in d-spacing, and smoothing will avoid adding unnecessary noise to the data. It might be necessary to experiment to find the correct smoothing parameter `sigma`. 

## Smooth vanadium

In [ ]:
from scipp.scipy.ndimage import gaussian_filter

norm_column_smooth = sc.DataGroup()

for i in range(6):
    norm_column_smooth[f'Column{i+1}'] = gaussian_filter(sc.values(norm_column['columns', i]), 
                                                         sigma=10)

pp.plot(norm_column_smooth, 
        title='Vanadium-Empty smoothed', 
        errorbars=False, 
        grid=True)

## Final reduced data

Now we can apply this normalisation to obtain the reduced data:

In [ ]:
pp.plot({f'Column {i}': si_dsp_calib_gp_hist['columns', sc.scalar(i, unit=None)] for i in range(1, 7)}, 
        grid=True,  title='Si histogrammed', errorbars=False)

In [ ]:
silicon_column_final = sc.DataGroup()
for i in range(6):
    silicon_column_final[f'Column{i+1}'] = si_dsp_calib_gp_hist['columns', i] / norm_column_smooth[f'Column{i+1}']
    
silicon_column_final

In [ ]:
pp.plot(silicon_column_final, grid=True,  title='Si/(Vana-Empty)', errorbars=False)

This evolution of the spectra as function of d-dspacing is because at the edge of each individual spectrum, the denominator `norm_column_smooth` becomes very small and the result of dividing becomes ill conditioned (and then NAN where the denominator is exactly zero). 

Since the d-spacing range is angle dependent, this happens at a different d-spacing for each spectrum. We need custom binning parameters for each spectrum. These can be calculated for each spectrum and are tablulated here: 

| column | average 2$\theta$ (deg) | d$_{min}$ (Å)| d$_{max}$ (Å)| number of bins|
|-------|-------------------------|--------------|------------|-------------|
| 1 | 150 |0.3 |2.22 | 2503
| 2 | 120 |0.33 |2.48 | 2523
| 3 | 110 |0.37 |2.86 | 230
| 4 | 90  |0.39 |3.05 | 231
| 5 | 60  |0.45 |3.86 | 2151
| 6 | 50  |0.55 |5.26 | 3926


Below we go back to the silicon and vanadium data with events and use new binning edges:

In [ ]:
dsp_edge_min = [0.30, 0.33, 0.37, 0.39, 0.45, 0.55]
dsp_edge_max = [2.22, 2.48, 2.86, 3.05, 3.86, 5.26]
dsp_nb_bins = [2504, 2523, 230, 231, 2151, 3926]

van_minus_empty = van_bin.bins.concatenate(-empty_bin)

silicon_column_final = sc.DataGroup()

for i in range(6):
    dsp_bins = sc.geomspace(dim='dspacing', 
                              unit='angstrom', 
                              start=dsp_edge_min[i],
                              stop=dsp_edge_max[i],
                              num=dsp_nb_bins[i]) 

    silicon_column_final[f'Column{i+1}'] = si_dsp_calib_gp_bin['columns', i].bin(dspacing=dsp_bins).hist() / gaussian_filter(
        sc.values(van_minus_empty['columns', i].bin(dspacing=dsp_bins).hist()), 
                                                         sigma=10)

In [ ]:
pp.plot({f'Column{i}': silicon_column_final[f'Column{i}'] for i in range(1,7)}, 
        errorbars=False,
        grid=True, title='Final Silicon')

Congratulations! Now that your diffraction data are correctly calibrated, normalised and focussed, they are ready to export to a Rietveld package for analysis!


**Bonus**: Without smoothing the Vanadium data, we get:

In [ ]:
silicon_column_final_bonus = sc.DataGroup()

for i in range(6):
    dsp_bins = sc.geomspace(dim='dspacing', 
                              unit='angstrom', 
                              start=dsp_edge_min[i],
                              stop=dsp_edge_max[i],
                              num=dsp_nb_bins[i]) 
    silicon_column_final_bonus[f'Column{i+1}'] = \
    si_dsp_calib_gp_bin['columns', i].bin(dspacing=dsp_bins).hist() / van_minus_empty['columns', i].bin(dspacing=dsp_bins).hist()

pp.plot({f'Column{i}': silicon_column_final_bonus[f'Column{i}'] for i in range(1,7)}, 
        errorbars=False,
        grid=True, title='Final Silicon without Vanadium smoothing')